# ライブラリインポート

In [1]:
import random

import numpy as np
from numpy.typing import NDArray

# 条件設定

In [2]:
# 都市間の距離行列(隣接行列)
# distances[i][j]は都市iからjまでの距離
distances: NDArray[np.int_] = np.array([
    [0, 2, 9, 10], # 都市0(この行)から都市1へは距離2、都市2へは距離9、都市3へは距離10です
    [1, 0, 6, 4],
    [15, 7, 0, 8],
    [6, 3, 12, 0]
])

# 関数

## 個体の表現
各個体は都市の訪問順序を表す。

In [3]:
def generate_individual(num_cities: int) -> list[int]:
    """ 都市の訪問順序をランダムに生成する """
    route = list(range(num_cities))
    random.shuffle(route)
    return route

## 適応度関数
訪問ルートの総距離の逆数とする。小さいほど適応度が高いことになる。

In [4]:
def fitness(route: list[int], distance_matrix: NDArray[np.int_]) -> float:
    """ ルートの総距離を計算し、適応度を返す """
    total_distance = 0
    for i in range(len(route)):
        total_distance += distance_matrix[route[i-1]][route[i]]
    return 1 / total_distance

# 選択

In [5]:
def tournament_selection(population: list[list[int]], scores: list[float], k: int=3):
    """ トーナメント選択を行う関数 """
    # トーナメントのサイズkでランダムに個体を選ぶ
    selection: list[tuple[list[int], float]] = random.sample(list(zip(population, scores)), k) # (polulation, score)のペアをランダムにk個取り出す
    # 適応度が最も高い個体を選ぶ
    selected = max(selection, key=lambda x: x[1])
    return selected[0]

## 交叉
順番を保持する必要がある。

In [6]:
def crossover(parent1: list[int], parent2: list[int]):
    """ 一様順序交叉を行う """
    child = [-1] * len(parent1)
    indices = sorted(random.sample(range(len(parent1)), 2))
    child[indices[0]:indices[1]] = parent1[indices[0]:indices[1]]
    
    fill_from = indices[1]
    for gene in parent2:
        if gene not in child:
            if fill_from >= len(parent1):
                fill_from = 0
            child[fill_from] = gene
            fill_from += 1
            
    return child

## 突然変異

In [7]:
def mutate(route: list[int], mutation_rate: float=0.02):
    """ 2-opt法による突然変異 """
    if random.random() < mutation_rate:
        i, j = sorted(random.sample(range(len(route)), 2))
        route[i:j+1] = reversed(route[i:j+1])
    return route

# メインループ

In [8]:
def genetic_algorithm(distance_matrix, population_size=100, generations=200, mutation_rate=0.02):
    """ 遺伝的アルゴリズムのメイン関数 """
    num_cities = len(distance_matrix)
    population = [generate_individual(num_cities) for _ in range(population_size)]
    best_route = None
    best_fitness = float('inf')

    for generation in range(generations):
        # 適応度を計算
        fitnesses = [fitness(individual, distance_matrix) for individual in population]
        
        # 次世代の個体群を選択
        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child = crossover(parent1, parent2)
            new_population.append(mutate(child, mutation_rate))
        
        population = new_population
        
        # 最良の解を更新
        current_best_fitness = min(fitnesses)
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_route = population[fitnesses.index(current_best_fitness)]
        
        # 進捗を表示
        print(f"Generation {generation}: Best Fitness = {1 / best_fitness}")

    return best_route, 1 / best_fitness


In [9]:
distance_matrix = np.array([
    [0, 2, 9, 10],
    [1, 0, 6, 4],
    [15, 7, 0, 8],
    [6, 3, 12, 0]
])
best_route, best_distance = genetic_algorithm(distance_matrix)
print("Best Route:", best_route)
print("Best Distance:", best_distance)

Generation 0: Best Fitness = 34.0
Generation 1: Best Fitness = 34.0
Generation 2: Best Fitness = 34.0
Generation 3: Best Fitness = 34.0
Generation 4: Best Fitness = 34.0
Generation 5: Best Fitness = 34.0
Generation 6: Best Fitness = 34.0
Generation 7: Best Fitness = 34.0
Generation 8: Best Fitness = 34.0
Generation 9: Best Fitness = 34.0
Generation 10: Best Fitness = 34.0
Generation 11: Best Fitness = 34.0
Generation 12: Best Fitness = 34.0
Generation 13: Best Fitness = 34.0
Generation 14: Best Fitness = 34.0
Generation 15: Best Fitness = 34.0
Generation 16: Best Fitness = 34.0
Generation 17: Best Fitness = 34.0
Generation 18: Best Fitness = 34.0
Generation 19: Best Fitness = 34.0
Generation 20: Best Fitness = 34.0
Generation 21: Best Fitness = 34.0
Generation 22: Best Fitness = 34.0
Generation 23: Best Fitness = 34.0
Generation 24: Best Fitness = 34.0
Generation 25: Best Fitness = 34.0
Generation 26: Best Fitness = 34.0
Generation 27: Best Fitness = 34.0
Generation 28: Best Fitness = 